In this notebook, we will train and test a word embedding model with Word2vec.

The model is trained on a sample corpus of English texts from Wikipedia.
In the first block, let's dowload the dataset in XML format and parse it to extract only the text of the short abstracts of each article.

In [ ]:
!wget https://dumps.wikimedia.org/enwiki/20220820/enwiki-20220820-abstract1.xml.gz

import gzip
import xml.etree.ElementTree as ET

with gzip.open('enwiki-20220820-abstract1.xml.gz') as xml_file:
    tree = ET.parse(xml_file)
root = tree.getroot()

documents = []
for doc in root:
    abstract = doc.find("abstract")
    if abstract.text and len(abstract.text)>100:
        documents.append(abstract.text)
print (len(documents))

We want to train the embeddings with sentences rather than documents, so we tokenize the texts with the [default tokenizer](https://www.nltk.org/api/nltk.tokenize.html) for English from [NLTK](https://www.nltk.org/).

We also apply some basic normalization, i.e., putting all the words in lowercase.


In [ ]:
!pip install nltk

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

sentences = []
for document in documents:
    words = [word.lower() for word in word_tokenize(document)]
    sentences.append(words)
        

Training a Word2vec model with Gensim is straightforward. Check the [official documentation](https://www.nltk.org/api/nltk.tokenize.html) to explore the options and hyperparameters.

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=sentences, 
    size=100, 
    window=5, 
    min_count=1, 
    workers=4)

model.save("word2vec.model")

Once the word embedding model is trained, a dictionary ("vw" for word vector) becomes available that contains a vector representation for each work in the vocabulary.

The embeddings are ndarray objects from [Numpy](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html).

In [ ]:
print (type(model.wv['rock']))
print (model.wv['rock'])

Gensim offers useful functions to manipulate word embeddings, such as finding the N words whose representations are closer in the geometric space to an input word.

In [ ]:
for word, similarity in model.wv.most_similar('rock', topn=10):
    print (f"{similarity:.2f} {word}")
